# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234239918173                   -0.50    8.0
  2   -7.250000794750       -1.80       -1.39    1.0
  3   -7.251238492585       -2.91       -2.01    3.0
  4   -7.251106170920   +   -3.88       -1.97    5.0
  5   -7.251327600800       -3.65       -2.62    4.0
  6   -7.251337868238       -4.99       -3.08    3.0
  7   -7.251338794456       -6.03       -4.14    2.0
  8   -7.251338796569       -8.68       -4.41    3.0
  9   -7.251338798641       -8.68       -5.09    3.0
 10   -7.251338798699      -10.23       -5.70    2.0
 11   -7.251338798704      -11.35       -6.11    3.0
 12   -7.251338798704      -12.25       -6.48    2.0
 13   -7.251338798705      -12.88       -7.11    3.0
 14   -7.251338798705      -15.05       -7.49    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03459097563919216
[ Info: Arnoldi iteration step 2: normres = 0.5144575658434767
[ Info: Arnoldi iteration step 3: normres = 0.50547851435476
[ Info: Arnoldi iteration step 4: normres = 0.6648923189668737
[ Info: Arnoldi iteration step 5: normres = 0.48514165703168716
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.58e-01, 7.40e-02, 3.65e-01, 2.67e-01, 1.57e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3766354142616258
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.87e-02, 1.10e-01, 3.20e-01, 1.10e-01, 1.13e-01)
[ Info: Arnoldi iteration step 7: normres = 0.08838205940267875
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.23e-03, 1.39e-02, 2.41e-02, 5.11e-02, 5.66e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10494535604776385
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.01e-04